In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

import sys
sys.path.append('../')
import bokeh_helpers as bh
from helpers import load_mat_data

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

output_notebook(hide_banner=True)

In [2]:
nch_file = "../Book-on-gm-ID-design-main/starter_kit/180nch.mat"
nch_data_df = load_mat_data(nch_file)
# nch_mat = loadmat(nch_file, squeeze_me=True)
# nch_data = nch_mat['nch']


Loading data from ../Book-on-gm-ID-design-main/starter_kit/180nch.mat
Found the following columns: ['ID', 'VT', 'GM', 'GMB', 'GDS', 'CGG', 'CGS', 'CGD', 'CGB', 'CDD', 'CSS', 'STH', 'SFL', 'INFO', 'CORNER', 'TEMP', 'VGS', 'VDS', 'VSB', 'L', 'W', 'NFING']


/Users/sean/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pandas/core/internals/construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [ ]:
# # I want to put the contents of 180nch.mat into a dataframe
# # I'm not super familiar with how .mat files are structured,
# # so this will take some exploration.

# # first, let's get column names from the dtypes of the numpy
# # array that we got from reading in the .mat file
# display(list(nch_data.dtype.fields.keys()))
# nch_data_cols = list(nch_data.dtype.fields.keys())

In [ ]:
# display(nch_data.item()[0])

In [ ]:
# # now, i think I can get the dataset, compressed down to a single
# # row (we'll expand it later), with this:

# nch_data_df = pd.DataFrame(nch_data.item())
# nch_data_df = nch_data_df.T

In [ ]:
# nch_data_df.columns = nch_data_cols

In [ ]:
# nch_data_df.columns

In [ ]:
# meas_cols = ['ID', 'VT', 'GM', 'GMB', 'GDS', 'CGG', 'CGS', 'CGD', 'CGB', 'CDD',
#        'CSS', 'STH', 'SFL']

In [ ]:
# # what do the arrays of measured xtor data look like?
# display(nch_data_df['ID'][0].shape)

# # and how long are each of the parameter sweeps?
# display(len(nch_data_df['L'][0]))
# display(len(nch_data_df['VSB'][0]))
# display(len(nch_data_df['VGS'][0]))
# display(len(nch_data_df['VDS'][0]))

# # Ok, so it looks like the outermost sweep (and the first
# # to explode) is 'L', then 'VDS', 'VGS', and 'VSB' last.

# df1 = nch_data_df.explode(meas_cols + ['L'], ignore_index=True)
# df2 = df1.explode(meas_cols + ['VGS'], ignore_index=True)
# df3 = df2.explode(meas_cols + ['VDS'], ignore_index=True)
# df4 = df3.explode(meas_cols + ['VSB'], ignore_index=True)

In [ ]:
# nch_data_df = df4.copy()
# nch_data_df.to_csv("../Book-on-gm-ID-design-main/starter_kit/180nch.csv")

In [4]:
# ok, now let's plot a few things to sanity check. We might've
# gotten VDS and VGS backward.

# create a plot for Ids vs. Vds across Vgs
caption = "Ids vs. Vds for BSIM4 NMOS"
x_axis_label = "Vds (V)"
y_axis_label = "Ids (A)"

ids_fig = bh.create_bokeh_plot(title = caption, 
                        x_axis_label = x_axis_label,
                        y_axis_label = y_axis_label,
                        )

# select a subset of the data
# plot_group = nch_data_df.groupby(['L', 'VSB', 'VGS']).get_group((0.180, 0.0, 1.8))
plot_group = nch_data_df.groupby(['L', 'VSB']).get_group((0.180, 0.0))
# data = ColumnDataSource(plot_group)
# display(plot_group[['VDS', 'ID']])
# ids_fig.line(x='VDS', y='ID', source=data, legend_label=f"vgs={vgs}")
# ids_fig.scatter(x='VDS', y='ID', source=data)

for vgs, vgs_group in plot_group.groupby('VGS'):
    data = ColumnDataSource(vgs_group)
    ids_fig.line(x='VDS', y='ID', source=data, legend_label=f"vgs={vgs}")
    ids_fig.scatter(x='VDS', y='ID', source=data)
    # display(vgs_group)
show(ids_fig)

Got it!!

In [5]:
nch_data_df['GMID'] = nch_data_df['GM'] / nch_data_df['ID']

In [6]:
# extra sanity check: for some length, check the gm/id vs. id curve

plot_group = nch_data_df.groupby(['L', 'VSB', 'VDS']).get_group((0.180, 0.0, 1.8))

# create a plot for Ids vs. Vds across Vgs
caption = "Gm/Id vs. VGS for Murmann's 180nm data"
x_axis_label = "VGS (V)"
y_axis_label = "Gm / Id (mohs)"
tooltips = [
    ('x', '$x'),
    ('y', '$y')
]

gmid_fig = figure(title = caption, 
                        x_axis_label = x_axis_label,
                        y_axis_label = y_axis_label,
                        x_axis_type = 'linear',
                        tooltips = tooltips
                        )

data = ColumnDataSource(plot_group)
# display(plot_group[['VDS', 'ID']])
gmid_fig.line(x='VGS', y='GMID', source=data,)
gmid_fig.scatter(x='VGS', y='GMID', source=data)

# for vgs, vgs_group in plot_group.groupby('VGS'):
#     data = ColumnDataSource(vgs_group)
#     gmid_fig.line(x='VDS', y='ID', source=data, legend_label=f"vgs={vgs}")
#     gmid_fig.scatter(x='VDS', y='ID', source=data)
#     # display(vgs_group)
show(gmid_fig)